This builds on the previous geeet-v3 notebook. This assumes we have an image collection of monthly ET images as assets in Earth Engine. All data pre-processing is done in the server, and will result in a CSV containing a chart of monthly ET at each combination of landcover, elevation, aspect, and slope based on our classification.

In [36]:
import ee

ee.Initialize()

# ----------------------------
# USER PARAMETERS
# ----------------------------
region = ee.FeatureCollection("projects/ee-redwall6152/assets/DC_bound")
start_year = 2013
end_year = 2025
months = list(range(4, 11))  # April–October
EXPORT_FOLDER = "GEEET_analysis"
EXPORT_PREFIX = "DryCreek_ET_zonal_nan"
ET_BAND = "ET_smoothed"
ET_ASSET_FOLDER = "projects/ee-redwall6152/assets/DC_ET_monthly"

# ----------------------------
# BUILD ET COLLECTION FROM FOLDER
# ----------------------------
def list_images_in_folder(folder_path):
    assets = ee.data.listAssets({"parent": folder_path})
    return [a['id'] for a in assets.get('assets', []) if a['type'] == 'IMAGE']

image_ids = list_images_in_folder(ET_ASSET_FOLDER)
if not image_ids:
    raise RuntimeError("No images found in ET asset folder.")
et_collection = ee.ImageCollection(image_ids).filterBounds(region)

# Use first ET image to define projection and scale
rep_et = ee.Image(et_collection.first()).select([ET_BAND])
et_proj = rep_et.projection()
et_scale = rep_et.projection().nominalScale()

# ----------------------------
# PREPARE TERRAIN AND NLCD (SNAPPED TO ET GRID)
# ----------------------------
dem = ee.Image('USGS/3DEP/10m').resample('bilinear') \
        .reproject(crs=et_proj.crs(), scale=et_scale).clip(region)

terrain = ee.Terrain.products(dem)
slope = terrain.select('slope')
aspect = terrain.select('aspect')

nlcd = ee.Image("USGS/NLCD_RELEASES/2021_REL/NLCD/2021") \
           .select('landcover') \
           .reproject(crs=et_proj.crs(), scale=et_scale).clip(region)

# ----------------------------
# CREATE CLASSES
# ----------------------------
# Aspect: 8 classes
aspect_class = aspect.divide(45).floor().clamp(0, 7).toInt()

# Elevation: 5 equal-interval bins
elev_stats = dem.reduceRegion(ee.Reducer.minMax(), region.geometry(), et_scale, bestEffort=True)
minElev = ee.Number(elev_stats.get('elevation_min'))
maxElev = ee.Number(elev_stats.get('elevation_max'))
elev_interval = maxElev.subtract(minElev).divide(5)
elev_class = dem.subtract(minElev).divide(elev_interval).floor().clamp(0, 4).toInt()

# Slope: 5 bins (0–4)
slope_class = slope.expression(
    "(s < 5) ? 0 : (s < 15) ? 1 : (s < 25) ? 2 : (s < 35) ? 3 : 4", {'s': slope}
).toInt()

# Zone ID encoding: aspect*1e6 + elev*1e4 + slope*1e2 + nlcd
zone_id = aspect_class.multiply(1e6).add(elev_class.multiply(1e4)) \
            .add(slope_class.multiply(1e2)).add(nlcd).rename('zone_id').toInt()

zone_id = zone_id.setDefaultProjection(et_proj)

# ----------------------------
# ZONAL STATS FUNCTION (WITH NaN FOR MISSING)
# ----------------------------
def stats_for_month(date_str):
    start = ee.Date(date_str)
    end = start.advance(1, 'month')
    
    imgs = et_collection.filterDate(start, end)
    has_imgs = imgs.size().gt(0)
    
    def compute():
        monthly = imgs.select([ET_BAND]).median().rename('ET').set('system:time_start', start.millis())
        monthly = monthly.setDefaultProjection(et_proj.crs(), None, et_scale)
        
        # Add zone band first for grouped reducer
        img_for_reduce = monthly.addBands(zone_id)
        
        # Grouped reducer: mean, median, stdDev, count
        base = ee.Reducer.mean().combine(ee.Reducer.median(), '', True) \
                               .combine(ee.Reducer.stdDev(), '', True) \
                               .combine(ee.Reducer.count(), '', True)
        grouped = base.group(groupField=1, groupName='zone_id')
        
        stats = img_for_reduce.reduceRegion(
            reducer=grouped,
            geometry=region.geometry(),
            scale=et_scale,
            maxPixels=1e13,
            tileScale=4
        )
        
        groups = ee.List(ee.Dictionary(stats).get('groups'))
        
        # If no groups, return empty FeatureCollection (will be NaN)
        def group_to_feature(g):
            gd = ee.Dictionary(g)
            z = ee.Number(gd.get('zone_id'))
            
            # Use ee.Algorithms.If to safely handle missing keys
            mean_v = ee.Algorithms.If(gd.contains('mean'), gd.get('mean'), None)
            median_v = ee.Algorithms.If(gd.contains('median'), gd.get('median'), None)
            std_v = ee.Algorithms.If(gd.contains('stdDev'), gd.get('stdDev'), None)
            count_v = ee.Algorithms.If(gd.contains('count'), gd.get('count'), None)
            
            aspect_val = z.divide(1e6).floor().toInt()
            elev_val = z.divide(1e4).mod(100).floor().toInt()
            slope_val = z.divide(1e2).mod(100).floor().toInt()
            nlcd_val = z.mod(100).toInt()
            
            props = {
                'datetime': start.format('YYYY-MM-01'),
                'zone_id': z,
                'aspect_class': aspect_val,
                'elev_class': elev_val,
                'slope_class': slope_val,
                'nlcd': nlcd_val,
                'mean_ET': mean_v,
                'median_ET': median_v,
                'std_ET': std_v,
                'count': count_v
            }
            return ee.Feature(None, props)
        
        return ee.FeatureCollection(groups.map(group_to_feature))
    
    # If no images, return empty FC (all missing)
    return ee.Algorithms.If(has_imgs, compute(), ee.FeatureCollection([]))

# ----------------------------
# BUILD FULL TIME SERIES
# ----------------------------
month_strings = [f"{y:04d}-{m:02d}-01" for y in range(start_year, end_year+1) for m in months]
all_fcs = [ee.FeatureCollection(stats_for_month(ds)) for ds in month_strings]

full_fc = ee.FeatureCollection(all_fcs).flatten()

# ----------------------------
# EXPORT TO DRIVE
# ----------------------------
task = ee.batch.Export.table.toDrive(
    collection=full_fc,
    description=EXPORT_PREFIX,
    folder=EXPORT_FOLDER,
    fileNamePrefix=EXPORT_PREFIX,
    fileFormat='CSV'
)
task.start()
print("✅ Export started. Check Tasks or your Drive folder.")
# To check status, run:
while task.status()['state'] in ['READY', 'RUNNING']:
            print(f"⏳")
            time.sleep(60)

status = task.status()
if status['state'] == 'COMPLETED':
            print(f"✅ Export complete")
else:
            print(f"❌ Export faileD: {status}")

✅ Export started. Check Tasks or your Drive folder.
⏳
⏳
✅ Export complete
